In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchaudio
import pandas as pd
import os
import gc
from tqdm import tqdm
from glob import glob
import time
import scipy
from functools import partial

import torch.jit as jit
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
from torchvision.models import get_model
import timm

import openvino as ov
import openvino.properties as props
import openvino.properties.hint as hints
from concurrent import futures

/home/cedric/miniconda3/envs/birds/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class Config:
    duration = 5
    sample_rate = 32000
    target_length = 500
    n_mels = 128
    n_fft = 1024
    window = 800
    audio_len = duration*sample_rate
    hop_length = audio_len // (target_length-1)
    fmin = 20
    fmax = 16000
    top_db = 80

    n_classes = 182
    n_channels = 1

    use_openvino = True
    multithreading = False
    checkpoint_dir = 'checkpoints/2024-05-29_08-53-40_128x500_mn20_as_fold-2'
    loss = 'crossentropy'
    ensemble_checkpoints = ['checkpoints/2024-05-23_00-55-30_256x256_convnextv2_tiny.fcmae_ft_in22k_in1k_fold-0',
                            'checkpoints/2024-05-23_00-55-30_256x256_convnextv2_tiny.fcmae_ft_in22k_in1k_fold-2',
                            'checkpoints/2024-05-23_00-55-30_256x256_convnextv2_tiny.fcmae_ft_in22k_in1k_fold-3'
                            ]
    ensemble_losses = ['crossentropy', 'crossentropy', 'crossentropy']

    standardize = False
    dataset_mean = [-16.8828]
    dataset_std = [12.4019]

In [10]:
def create_frames(waveform, duration=5, sr=32000):
    frame_size = int(duration * sr)
    surplus = waveform.size(-1)%frame_size
    if surplus > 0:
        waveform = waveform[:, :-surplus]
    frames = waveform.view(-1, 1, frame_size)
    return frames

class AudioDatasetInference(Dataset):
    def __init__(
            self, 
            files,
            targets = None, 
            n_classes = 182,
            duration = 5,
            sample_rate = 32000,
            target_length = 384,
            n_mels = 128,
            n_fft = 2028,
            window = 2028,
            hop_length = None,
            fmin = 20,
            fmax = 16000,
            top_db = 80,
            standardize=True,
            mean=None,
            std=None
            ):
        super(AudioDatasetInference, self).__init__()
        self.files = files
        self.targets = targets
        self.n_classes = n_classes
        self.duration = duration
        self.sample_rate = sample_rate
        self.audio_len = duration*sample_rate
        self.target_length = target_length
        self.n_mels = n_mels
        self.n_fft = n_fft
        self.window = window
        self.hop_length = self.audio_len // (target_length-1) if not hop_length else hop_length
        self.fmin = fmin
        self.fmax = fmax
        self.top_db = top_db
        self.standardize = standardize

        self.to_mel_spectrogramn = nn.Sequential(
            torchaudio.transforms.MelSpectrogram(self.sample_rate, n_fft=self.n_fft, win_length=self.window,  
                                                 hop_length=self.hop_length, n_mels=self.n_mels, 
                                                 f_min=self.fmin, f_max=self.fmax),
            torchaudio.transforms.AmplitudeToDB(top_db=self.top_db)
        )
        if mean is not None and std is not None:
            self.to_mel_spectrogramn.append(v2.Normalize(mean=mean, std=std))

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        file = self.files[idx]
        waveform, sr = torchaudio.load(file)
        frames = create_frames(waveform)
        spec = self.to_mel_spectrogramn(frames)
        # Standardize
        if self.standardize:
            spec = (spec - spec.mean()) / spec.std()

        # expand to 3 channels for imagenet trained models
        spec = spec.expand(-1,Config.n_channels,-1,-1)

        if self.targets is not None:
            label = torch.tensor(self.targets[idx])
            return spec, label
        else:
            return spec, file

In [11]:
base_dir = 'data'
train_dir = base_dir + '/train_audio/'
test_dir = base_dir + '/test_soundscapes/'
unlabeled_dir = base_dir + '/unlabeled_soundscapes/'

class_names = sorted(os.listdir(train_dir))
n_classes = len(class_names)
class_labels = list(range(n_classes))
label2name = dict(zip(class_labels, class_names))
name2label = {v:k for k,v in label2name.items()}

In [12]:
test_paths = glob(base_dir + '/test_soundscapes/*ogg')
if len(test_paths)==0:
    test_paths = glob(base_dir + '/unlabeled_soundscapes/*ogg')[:10]
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df.head()

,filepath
0,data/unlabeled_soundscapes/646255149.ogg
1,data/unlabeled_soundscapes/1171835482.ogg
2,data/unlabeled_soundscapes/1590789246.ogg
3,data/unlabeled_soundscapes/115033522.ogg
4,data/unlabeled_soundscapes/1971688290.ogg


In [13]:
test_dataset = AudioDatasetInference(
    test_df['filepath'].values, 
    targets=None, 
    n_classes=Config.n_classes,
    duration=5,
    sample_rate=Config.sample_rate,
    target_length=Config.target_length,
    n_mels=Config.n_mels,
    n_fft=Config.n_fft,
    window=Config.window,
    hop_length=Config.hop_length,
    fmin=Config.fmin,
    fmax=Config.fmax,
    top_db=Config.top_db,
    standardize=Config.standardize,
    mean=Config.dataset_mean,
    std=Config.dataset_std
    )

In [14]:
if Config.multithreading:
    def predict(dataset, model, loss):
        ids = []
        preds = np.empty(shape=(0, n_classes), dtype='float32')
        #output_layer = model.output(0)
        infer_request = model.create_infer_request()
        if loss == 'crossentropy':
            final_activation = partial(scipy.special.softmax, axis=1)
        elif loss == 'bce':
            final_activation = scipy.special.expit

        specs, file = dataset[0]
        for i in range(1, len(dataset)):
            infer_request.start_async([specs])
            specs, file = dataset[i]
            filename = file.split('/')[-1][:-4]

            #outs = model([specs])[output_layer]
            infer_request.wait()
            outs = infer_request.get_output_tensor(0).data
            outs = final_activation(outs)

            frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(specs))]
            ids += frame_ids

            preds = np.concatenate([preds, outs], axis=0)

        return preds, ids

    def run_prediction(data_loader, model_id):
        core = ov.Core()
        checkpoint_ov = Config.ensemble_checkpoints[model_id] + '/checkpoint.xml'
        loss = Config.ensemble_losses[model_id]
        config = {hints.performance_mode: hints.PerformanceMode.THROUGHPUT}
        model = core.compile_model(checkpoint_ov, "CPU", config)
        
        preds, ids = predict(data_loader, model, loss)
        del core, model, loss
        gc.collect()
        
        print(f"Done model {model_id}")
        return preds, ids

    def helper(inputs):
        return run_prediction(inputs[0], inputs[1])


    start=time.time()
    
    audios = [(test_dataset, model_id) for model_id in range(len(Config.ensemble_checkpoints))]
    ensemble_preds = []
    with futures.ThreadPoolExecutor(max_workers=len(Config.ensemble_checkpoints)) as executor:
        for preds, ids in executor.map(helper, audios):
            ensemble_preds.append(preds)
    ensemble_preds = np.array(ensemble_preds)
    ensemble_preds = np.mean(ensemble_preds, axis=0)
    #ensemble_preds = (ensemble_preds**2).mean(axis=0) ** 0.5

    print(time.time()-start)

In [15]:
if Config.use_openvino:
    start=time.time()
    
    checkpoint_ov = Config.checkpoint_dir + '/checkpoint.xml'
    config = {hints.performance_mode: hints.PerformanceMode.THROUGHPUT}
    core = ov.Core()
    model = core.compile_model(checkpoint_ov, "CPU", config)
    #infer_request = model.create_infer_request()
    output_layer = model.output(0)

    ids = []
    preds = np.empty(shape=(0, n_classes), dtype='float32')
    if Config.loss == 'crossentropy':
        final_activation = partial(scipy.special.softmax, axis=1)
    elif Config.loss == 'bce':
        final_activation = scipy.special.expit

    test_iter = tqdm(range(len(test_dataset)))
    for i in test_iter:
        specs, file = test_dataset[i]
        filename = file.split('/')[-1][:-4]

        outs = model([specs])[output_layer]
        #outs = infer_request.infer([specs])[0]
        outs = final_activation(outs)

        frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(specs))]
        ids += frame_ids

        preds = np.concatenate([preds, outs], axis=0)

    print(time.time()-start)

100%|██████████| 10/10 [00:17<00:00,  1.76s/it]

17.917014598846436


In [12]:
if not Config.use_openvino and not Config.multithreading:
    device = torch.device('cpu')
    checkpoint_name = Config.checkpoint_dir + '/checkpoint.pth'
    model = src.models.BasicClassifier(Config.n_classes, pretrained=False, model_name=Config.model_name).to(device)
    checkpoint = torch.load(checkpoint_name, map_location='cpu')
    model.load_state_dict(checkpoint['model'])
    model = torch.jit.optimize_for_inference(torch.jit.script(model.eval()))

    ids = []
    preds = np.empty(shape=(0, n_classes), dtype='float32')

    test_iter = tqdm(range(len(test_dataset)))
    for i in test_iter:
        specs, file = test_dataset[i]
        filename = file.split('/')[-1][:-4]
        specs = specs.to(device)

        with torch.no_grad():
            outs = model(specs)
            if Config.loss == 'crossentropy':
                outs = nn.functional.softmax(outs, dim=1).detach().cpu().numpy()
            elif Config.loss == 'bce':
                outs = outs.sigmoid().detach().cpu().numpy()

        frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(specs))]
        ids += frame_ids

        preds = np.concatenate([preds, outs], axis=0)

100%|██████████| 10/10 [03:01<00:00, 18.11s/it]


In [18]:
# Submit prediction
pred_df = pd.DataFrame(ids, columns=['row_id'])
pred_df.loc[:, class_names] = preds
pred_df.to_csv('submission.csv',index=False)

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,XC756601_5,0.005847,0.011639,0.002512,0.000110,0.019070,0.000015,0.000609,0.000143,0.000069,...,0.000811,0.000281,0.001206,0.000926,0.005042,0.010253,0.000052,0.000306,0.000032,0.004853
1,XC756601_10,0.008243,0.007165,0.001763,0.000092,0.018658,0.000011,0.000512,0.000136,0.000104,...,0.000528,0.000333,0.002916,0.001131,0.004920,0.010568,0.000061,0.000224,0.000028,0.005552
2,XC756601_15,0.010177,0.009778,0.003066,0.000166,0.017798,0.000020,0.000657,0.000119,0.000143,...,0.000895,0.000277,0.001253,0.001104,0.007026,0.008240,0.000074,0.000242,0.000039,0.006269
3,XC756601_20,0.006208,0.006755,0.002717,0.000150,0.014288,0.000024,0.000561,0.000175,0.000171,...,0.000782,0.000253,0.001888,0.001559,0.007132,0.009607,0.000056,0.000403,0.000023,0.012148
4,XC756601_25,0.006516,0.006837,0.001713,0.000164,0.020643,0.000016,0.000494,0.000309,0.000087,...,0.000877,0.000263,0.001813,0.001479,0.005192,0.010191,0.000060,0.000307,0.000018,0.007364
